In [1]:
import numpy as np
gt = np.load("./groundtruth_36_tensor(0.0089).npy")
pr = np.load("./prediction_36_tensor(0.0089).npy")
print(np.shape(gt), np.shape(pr)) #psl channel

(197, 6, 1, 2562) (197, 1, 1, 2562)


In [2]:
def mse(a1, a2):
    se = np.square(np.subtract(a1,a2))
    mse = se.mean()
    return mse

In [3]:
ch =['tas', 'psl', 'pr']
i=2 #channel
print(ch[i],": mse = ",mse(gt, pr))

pr : mse =  0.008902849933213747


In [4]:
print(np.shape(gt), np.shape(pr))

(197, 6, 1, 2562) (197, 1, 1, 2562)


In [6]:
from ../interpolate import *

SyntaxError: invalid syntax (3922298085.py, line 1)

In [19]:
import cartopy.crs as ccrs
import xarray as xr

path = "./data/"
inn = xr.open_dataset(path+"Processed_CESM2_r1i1p1f1_historical_Input.nc")
inn['time'][:5]

<xarray.DataArray 'time' (time: 5)>
array([cftime.DatetimeNoLeap(1850, 1, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 14, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 4, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 5, 15, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 1850-05-15 12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    standard_name:  time
    title:          time
    type:           double

In [35]:
inn.variables

Frozen({'time': <xarray.IndexVariable 'time' (time: 1980)>
array([cftime.DatetimeNoLeap(1850, 1, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 14, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 15, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(2014, 10, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 11, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 15, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Attributes:
    axis:           T
    bounds:         time_bnds
    standard_name:  time
    title:          time
    type:           double, 'lat': <xarray.IndexVariable 'lat' (lat: 192)>
array([-90.      , -89.057592, -88.115183, -87.172775, -86.230366, -85.287958,
       -84.34555 , -83.403141, -82.460733, -81.518325, -80.575916, -79.633508,
       -78.691099, -77.748691, -76.806283, -75.863874, -74.921466, -73.979058,
       -73.036649, -72.09

In [36]:
inn.cres

<xarray.DataArray 'cres' (time: 1980, lat: 192, lon: 288)>
[109486080 values with dtype=float32]
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:    Cloud radiative effect in shortwave
    units:        W m-2
    Description:  (rsdt - rsut) - (rsdt - rsutcs)

In [28]:
lon = np.asarray(inn.lon);lat = np.asarray(inn.lat)

In [39]:
np.shape(lon), np.shape(inn.cres)

((288,), (1980, 192, 288))

In [40]:
lon, lat, interpolated_value = interpolate_SphereIcosahedral( 6, inn.cres[0], list(lon),list(lat))

number of vertices for level 6 (2**6)is 40962


In [10]:
lon=np.load("lon.npy")

In [11]:
lat = np.load("lat.npy")

In [12]:
def visualize_2d_all(gt, pr, longitude, latitude, title, export_path=None):
    """Visualize the data on a 2D map

    Args:
        x (numpy.array): numpy array with data the size of the longitude and latitude
        longitude (numpy.array): longitude coordinates
        latitude (numpy.array): latitude coordinates
        export_path (string): path and name for saving
    """

    fig = plt.figure(figsize=(70, 10))
    ax = fig.add_subplot(1, 3, 1, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()

    plt.subplot(1, 3, 1)
    plt.scatter(longitude, latitude, s=20, c=gt, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.clim(0, 1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.title("ground truth "+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)

    ax = fig.add_subplot(1, 3, 2, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    plt.subplot(1, 3, 2)
    plt.scatter(longitude, latitude, s=20, c=pr, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.clim(0, 1)
    plt.title("prediction "+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)
    
    ax = fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    plt.subplot(1, 3, 3)
    plt.scatter(longitude, latitude, s=20, c=gt-pr, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.clim(-1, 1)
    plt.title("error (gt-pr)"+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)
    if export_path:
        plt.savefig(export_path)
        plt.clf()
        plt.cla()
        plt.close()

    else:
        plt.show()

In [14]:
import os
os.mkdir("./results_convlstm_tas/fig/")
ch =['tas', 'psl', 'pr']

In [15]:
def mse(a1, a2):
    se = np.square(np.subtract(a1,a2))
    mse = se.mean()
    return mse

In [20]:
for i in range(198):
    print(np.shape(gt), np.shape(pr))
    error = mse(gt[i,:,0], pr[i,:,0])
    print(error)
    visualize_2d_all(gt[i,:,0],pr[i,:,0], lon, lat, ch[0]+" "+str(i), export_path="./results_convlstm_tas/fig/"+ch[0]+"_"+str(i)+"_"+str(error)+".png")


(19, 40962, 1) (19, 40962, 1)
0.012098899300066586
(19, 40962, 1) (19, 40962, 1)
0.025846023067747546
(19, 40962, 1) (19, 40962, 1)
0.01437540956160075
(19, 40962, 1) (19, 40962, 1)
0.014407415326646876
(19, 40962, 1) (19, 40962, 1)
0.02082624119664431
(19, 40962, 1) (19, 40962, 1)
0.017510993010442254
(19, 40962, 1) (19, 40962, 1)
0.016399424255760048
(19, 40962, 1) (19, 40962, 1)
0.02338557602891896
(19, 40962, 1) (19, 40962, 1)
0.021418378352359546
(19, 40962, 1) (19, 40962, 1)
0.012687594204933338
(19, 40962, 1) (19, 40962, 1)
0.019866347628718105
(19, 40962, 1) (19, 40962, 1)
0.02542885910651107
(19, 40962, 1) (19, 40962, 1)
0.016537266148718966
(19, 40962, 1) (19, 40962, 1)
0.020959181864132988
(19, 40962, 1) (19, 40962, 1)
0.012824438099597178
(19, 40962, 1) (19, 40962, 1)
0.011744938988197692
(19, 40962, 1) (19, 40962, 1)
0.01235516261029293
(19, 40962, 1) (19, 40962, 1)
0.012925605987706545
(19, 40962, 1) (19, 40962, 1)
0.01328864038406425
(19, 40962, 1) (19, 40962, 1)


IndexError: index 19 is out of bounds for axis 0 with size 19

In [21]:
def visualize_2d(x, longitude, latitude, title, export_path=None):
    """Visualize the data on a 2D map

    Args:
        x (numpy.array): numpy array with data the size of the longitude and latitude
        longitude (numpy.array): longitude coordinates
        latitude (numpy.array): latitude coordinates
        export_path (string): path and name for saving
    """

    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    ax.set_global()
    ax.coastlines()

    plt.scatter(longitude, latitude, s=20, c=x, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.title(title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"))
    if export_path:
        plt.savefig(export_path)
        plt.clf()
        plt.cla()
        plt.close()

    else:
        plt.show()

In [31]:
dt = np.load("./results_convlstm_tas/groundtruth_0_tensor(0.0175).npy")


In [32]:
np.shape(dt)

(19, 24, 1, 40962)

In [33]:
dt=np.reshape(dt[0,:,0,:],[24, 40962,1])
print(np.shape(dt))

(24, 40962, 1)


In [34]:
for i in range(24):
    visualize_2d(dt[i,:,:],lon, lat, "time_"+str(i), export_path="./results_convlstm_tas/fig/time_"+str(i)+".png")
